# 14. 데이터 불균형

데이터 불균형이란 데이터 세트 내의 클래스의 분포가 불균형한 것을 의미한다. 데이터 불균형은 특정 클래스에 과적합 되는 현상을 유발할 수 있기 때문에 반드시 해결해야 하는 문제다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd/content/drive/MyDrive/deeplearningbro/pytorch

## 14.1 Weighted Random Sampling
SGD base를 쓰기 때문에 일정한 배치가 들어옴 -> 전체 데이터는 불균형하더라도 배치를 균형데이터로 뽑는 것 (각 클래스들의 데이터들이 같은 확률로 뽑히게끔)

가중 샘플링 방식은 확률적 샘플링이기 때문에 전체 데이터 사용이 보장이 안 됩니다. 그럼에도 균형된 배치를 사용할 수 있기 때문에 랜덤 샘플링 보다는 더 좋은 성능이 이끌어 냅니다. 아주 오래 전에 저도 실제 연구에서 고민했던 질문이라 반갑습니다. 제가 시도 했던 방법은 크게 다음과 같습니다.

1. 가중 샘플링을 사용하되 데이터를 더 많이 활용할 수 있도록 에폭 수를 늘려 샘플링을 보다 많이 하게 한다.

2. 클래스마다 데이터 증식의 개수를 다르게 하여 불균형을 줄여준 뒤 무작위 샘플링을 진행한다. (oversampling)

3. 가장 수가 작은 클래스 기준으로 다른 클래스를 부분집합으로 나누어 매 에폭마다 돌아가면서 사용한다. (변형 된 undersampling)

4. 클래스가 가장 큰 데이터에 대해서 데이터 분석을 하여 일부를 추출하여 학습에 사용한다. 즉, 오히려 큰 클래스에 대해서 데이터가 덜 사용 됨을 인정하고 중요하다고 생각하는 데이터만 주입한다. (데이터 분석 후 가중 샘플링)

저의 경우 4번이 가장 큰 효과가 있었습니다

In [5]:
# 각 클래스의 비율을 정하여 뽑힐 확률에 대한 가중치를 산정한다.
import torch
from torch.utils.data import DataLoader
import torchvision
import numpy as np

def make_weights_for_balanced_classes(img, nclasses):

    labels = []
    for i in range(len(img)):
        labels.append(img[i][1])

    label_array = np.array(labels)
    total = len(labels)

    count_list = []
    for cls in range(nclasses):
        count = len(np.where(label_array == cls)[0])
        count_list.append(total/count)

    weights = []
    for label in label_array:
        weights.append(count_list[label])

    return weights


In [6]:
!pwd
../input/dl-class/class

In [9]:
trainset.imgs
trainset.classes

In [7]:
trainset = torchvision.datasets.ImageFolder(root='../input/dl-class/class', transform=transf) # Custom 데이터 세트 불러오기

weights = make_weights_for_balanced_classes(trainset.imgs, len(trainset.classes)) # 가중치 계산
weights = torch.DoubleTensor(weights) # 구한 weight를 텐서로 변환
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) # 샘플링 방법 정의

trainloader = DataLoader(trainset, batch_size=16, sampler=sampler) # 데이터 로더 정의

## 14.2 Weighted Loss Function

각각의 class마다 loss가 계산될 때 가중치를 곱해주는 방법
- ex : 10개의 클래스가 있다고 했을 때, loss 계산할 때 가장 큰 값에는 작은 값을 곱해주고 가장 작은 값에는 큰 값을 곱해줌으로써 데이터의 개수마다 loss 크기를 다르게 설정해줌

In [10]:
import torch.nn as nn
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_ins = [40,45,30,62,70,153,395,46,75,194]
weights = [1-(x/sum(num_ins)) for x in num_ins]
class_weights = torch.FloatTensor(weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)

## 14.3 Data Augmentation

Overfitting 방지를 위해서 쓰이지만 Data Imbalance 이슈에서도 쓰일 수 있음.

이미지를 더 늘리는 방법

In [14]:
import torchvision.transforms as tr
import PIL

transf = tr.Compose([tr.ToPILImage(), tr.RandomCrop(60), tr.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                 tr.RandomHorizontalFlip(),
                 tr.RandomRotation(10, resample=PIL.Image.BILINEAR),
                 tr.ToTensor()
                 ])

## 14.4 Confusion Matrix

직접적인 해결 방법은 아닌데, 학습을 마치고 나온 실제 값과 예측 값을 가지고 Matrix을 그려보는 것
- matrix를 그려봤을 때 class 중에서 어떤 클래스가 많이 맞췄고, 적게 맞췄는지를 파악해서 특정 클래스에 가중치를 더 줄 수 있고, 특정 클래스에 aug를 더 할 수도 있고, 이런 식으로 결과를 보고 다음 액션을 취할 수 있게 해준다.

In [15]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

actual = [1,1,1,1,1,1,0,0,0,0,0,2,2,2,2,2]
prediction = [1,1,1,0,1,1,0,0,0,1,0,2,2,2,1,1]
c_mat = confusion_matrix(actual, prediction) # 실제 라벨, 예측값
plt.figure(figsize = (8,6))
sns.heatmap(c_mat, annot=True, fmt="d", cmap='Blues',linewidths=.5)
b, t = plt.ylim() 
b += 0.5 
t -= 0.5 
plt.ylim(b, t) 
plt.savefig('confusion_matrix.png')
plt.show()

